In [1]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from sklearn import preprocessing
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import classification_report
from annoy import AnnoyIndex
import numpy as np
import pickle

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

enc = preprocessing.OneHotEncoder()
enc.fit(y_train)
y_train = enc.transform(y_train).toarray()

print("cifar loaded!")

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
cifar loaded!


In [2]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
lookup = np.argmax(y_train[:, :], axis=1)
lookup_test = y_test.flatten()

In [3]:
# def calculate(idx, kind):
#     vector_name = "";
#     if kind=="cnn":
#         vector_name = "features"
#     elif kind=="simple":
#         vector_name = "vectors-simple"
#     elif kind=="sparse":
#         vector_name = "vectors-sparse"
#     elif kind=="deep":
#         vector_name = "vectors-deep"
#     elif kind=="denoising":
#         vector_name = "vectors-denoising"
    
#     pickle_in = open("model-features/cbir-resnet-" + vector_name + ".pickle", "rb")
#     vectors = pickle.load(pickle_in)
#     index = AnnoyIndex(vectors.shape[1], metric='angular')
#     for i in range(vectors.shape[0]):
#         index.add_item(i, vectors[i,:].tolist())
#     index.build(20)
#     results = index.get_nns_by_item(idx, 5000)
#     t = np.ones(5000).astype(float)
#     p = np.array([lookup[x] == lookup[idx] for x in results]).astype(float)
#     result = [round(f1_score(t, p),3), round(precision_score(t, p),3), round(recall_score(t, p),3)]
#     return result

In [3]:
def calculate(idx, kind):
    vector_name = "";
    if kind=="cnn":
        vector_name = "features"
    elif kind=="simple":
        vector_name = "vectors-simple"
    elif kind=="sparse":
        vector_name = "vectors-sparse"
    elif kind=="deep":
        vector_name = "vectors-deep"
    elif kind=="denoising":
        vector_name = "vectors-denoising"
    
    pickle_features = open("pickle-test/cbir-resnet-" + vector_name + "-test.pickle", "rb")
    features = pickle.load(pickle_features)
    index = AnnoyIndex(features.shape[1], metric='angular')
    for i in range(features.shape[0]):
        index.add_item(i, features[i,:].tolist())
    index.build(20)
#     results = index.get_nns_by_item(idx, 5000))
    results = index.get_nns_by_vector(features[idx], 1000)
    t = np.ones(1000).astype(float)
    p = np.array([lookup_test[x] == lookup_test[idx] for x in results]).astype(float)
#     result = [round(f1_score(t, p),3), round(precision_score(t, p),3), round(recall_score(t, p),3)]
    result = round(accuracy_score(t, p),3)
    return result

In [4]:
from timeit import default_timer as timer
categories = y_test.flatten()
result = []
for y in range(10):
    cnn = []
    simple = []
    sparse = []
    deep = []
    denoising = []
    count = 0
    idx_cat = [i for i, e in enumerate(categories) if e == y]
    start = timer()
    for i in idx_cat:
        count += 1
        cnn.append(calculate(i, "cnn"))
        simple.append(calculate(i, "simple"))
        sparse.append(calculate(i, "sparse"))
        deep.append(calculate(i, "deep"))
        denoising.append(calculate(i, "denoising"))
#     data = {
#         "cnn": np.sum(cnn, axis = 0),
#         "simple": np.sum(simple, axis = 0),
#         "sparse": np.sum(sparse, axis = 0),
#         "deep": np.sum(deep, axis = 0),
#         "denoising": np.sum(denoising, axis = 0),
#     }
    data = {
        "cnn": cnn,
        "simple": simple,
        "sparse": sparse,
        "deep": deep,
        "denoising": denoising,
    }
    result.append(data)
    end = timer()
    print(y, " ", round(end - start), "s.")
print("Done calculate !!!")

0   1174 s.
1   1163 s.
2   1166 s.
3   1167 s.
4   1170 s.
5   1168 s.
6   1168 s.
7   1167 s.
8   1167 s.
9   1169 s.
Done calculate !!!


In [25]:
arr_cnn = []
arr_simple = []
arr_sparse = []
arr_deep = []
arr_denoising = []
for arr in result:
    arr_cnn.append(arr['cnn'])
    arr_simple.append(arr['simple'])
    arr_sparse.append(arr['sparse'])
    arr_deep.append(arr['deep'])
    arr_denoising.append(arr['denoising'])
mean_cnn = np.mean(np.array(arr_cnn))
mean_simple = np.mean(np.array(arr_simple))
mean_sparse = np.mean(np.array(arr_sparse))
mean_deep = np.mean(np.array(arr_deep))
mean_denoising = np.mean(np.array(arr_denoising))
print("                    F1-Score Precision    Recall")
print("cnn              : ", mean_cnn)
print("cnn-simple-ae    : ", mean_simple)
print("cnn-sparse-ae    : ", mean_sparse)
print("cnn-deep-ae      : ", mean_deep)
print("cnn-denoising-ae : ", mean_denoising)

                    F1-Score Precision    Recall
cnn              :  0.7139766000000001
cnn-simple-ae    :  0.7247149
cnn-sparse-ae    :  0.7262626
cnn-deep-ae      :  0.7052929
cnn-denoising-ae :  0.7214618


In [7]:
00000000000000000000000000000000000023.1.sum_cnn = np.sum(cnn, axis = 0)
print(sum_cnn)
sum_simple = np.sum(simple, axis = 0)
print(sum_simple)
sum_sparse = np.sum(sparse, axis = 0)
print(sum_sparse)
sum_deep = np.sum(deep, axis = 0)
print(sum_deep)
sum_denoising = np.sum(denoising, axis = 0)
print(sum_denoising)
sum_cnn = np.sum(cnn, axis = 0)
print(sum_cnn)

[0.998 1.    0.996]
[0.998 1.    0.996]
[0.998 1.    0.996]
[0.997 1.    0.995]
[0.998 1.    0.996]


In [12]:
arr = [[5,2,3],[5,2,3],[5,2,3]]
sum_arr = np.sum(arr, axis = 0)
print(sum_arr)
print(sum_arr[0]/3)

[15  6  9]
5.0
